# Sudan States, South Sudan States Sorting

In [ ]:
!pip install plotly==4.9.0

## Imports

In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
import plotly.graph_objects as go

In [ ]:
from IPython.display import HTML

Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Open plot in another tab

In [ ]:
# import plotly.io as pio
# pio.renderers.default = 'firefox'

## Load Sudan States

In [ ]:
downloaded = drive.CreateFile({'id':"1OOtLcYtFCDrFWNZxFCf1_RW4YWedyCj3"})   # replace the id with id of file you want to access
downloaded.GetContentFile('SD_SSD_States.geojson') 

In [ ]:
sudan = json.load(open("SD_SSD_States.geojson", "r"))

In [ ]:
state_id_map = {}
for feature in sudan["features"]:
    feature["id"] = feature["properties"]["StateNameAr"]
    state_id_map[feature["properties"]["StateID"]] = feature["id"]

## Load Sudan Data

### Static Data

In [ ]:
GSID = '13ot0rYLxegd-JxNBXjcF1obxwPzhl99sKSGICJpINiY'
WSN = 'StatesPoints'
URL = f'https://docs.google.com/spreadsheets/d/{GSID}/gviz/tq?tqx=out:csv&sheet={WSN}'

In [ ]:
df = pd.read_csv(URL)

### Animated Data

In [ ]:
GSID = '13ot0rYLxegd-JxNBXjcF1obxwPzhl99sKSGICJpINiY'
WSN = 'StatesPointsTime'
URL = f'https://docs.google.com/spreadsheets/d/{GSID}/gviz/tq?tqx=out:csv&sheet={WSN}'

In [ ]:
df = pd.read_csv(URL)

### Attatch an ID Column from 'geojson file' Into the DF w/ State's Name

In [ ]:
df['ID'] = df["الولاية"].apply(lambda x: list(state_id_map.keys())[list(state_id_map.values()).index(x)])

### Create Separate Data for Sudan/South Sudan

In [ ]:
dfsd = df.copy()

In [ ]:
# Clear South Sudan's ID's:
for i in range(0,len(df)):
    if df[i:i+1]['ID'][i].startswith('SS'):
        dfsd['الولاية'][i:i+1] = ''

In [ ]:
dfss = df.copy()

In [ ]:
# Clear Sudan's ID's:
for i in range(0,len(df)):
    if df[i:i+1]['ID'][i].startswith('SS') is False:
        dfss['الولاية'][i:i+1] = ''

## Plotting

### 2D Plots

In [ ]:
# df["النقاط"].plot()

### Choropleth Plot

#### Static

##### Sudan

In [ ]:
######################
fig = px.choropleth(
    dfsd,
    locations='الولاية', # From DF --> GeoJson id map
    geojson=sudan,
    color="النقاط",
    title="ترتيب الولايات حسب الجاهزية",
    color_continuous_scale=px.colors.diverging.RdYlGn,
).update_geos(fitbounds="locations", visible=False).update_traces(hovertemplate=None).update_layout(
font={"color": "rgb(64,64,64)",
      "family": "Times New Roman",
      "size": 20},
titlefont={"color": "rgb(64,64,64)",
           "family": "Times New Roman",
           "size": 40}
)
HTML(fig.to_html())
######################

In [ ]:
# Create Data for Top 3 States
dfsd_top = dfsd.copy()
# Clear points less than top 3:
for i in range(0,len(dfsd)):
    if dfsd[i:i+1]["النقاط"][i] < dfsd.nlargest(3, "النقاط")["النقاط"].min():
        dfsd_top = dfsd_top.drop([i])

In [ ]:
dfsd_top = dfsd_top.sort_values('النقاط',ascending=False)

In [ ]:
# Star Top 3 States:
HTML(fig.add_scattergeo(
    locations=dfsd_top["الولاية"],
    geojson=sudan,
    text=dfsd_top["الولاية"],
    textfont = {"color": "rgb(32,32,32)",
                "family": "Arial, Bold",
                "size": 20},
    textposition="bottom center",
    marker= dict(size=[30,25,20],
                 color=['Gold', 'Silver', 'Brown'],
                 opacity=1,
                 symbol='star'
                ),
    mode ="markers+text",
    showlegend=False,
    hoverinfo='skip'
).to_html())

##### Add South Sudan

In [ ]:
# Make South Sudan Points Nuetral
for i in range(0,len(df)):
    if dfss[i:i+1]['ID'][i].startswith('SS') is True:
        dfss["النقاط"][i:i+1] = (df["النقاط"].min() + df["النقاط"].max())/2

In [ ]:
HTML(fig.add_choropleth(
    autocolorscale=False,
    z=dfss["النقاط"],
    locations=dfss["الولاية"],
    locationmode='geojson-id',
    geojson=sudan,
    showscale=False,
    marker=dict(opacity=0.5),
    hoverinfo='location'
).update_traces(name='').to_html())

In [ ]:
# fig.write_html("drive/My Drive/SD_SSD_States_Static.html")

#### Animation

##### Sudan

In [ ]:
######################
fig = px.choropleth(
    dfsd,
    animation_frame='Time',
    locations='الولاية',
    geojson=sudan,
    color="النقاط",
    title="ترتيب الولايات حسب الجاهزية",
    color_continuous_scale=px.colors.diverging.RdYlGn
).update_geos(fitbounds="locations", visible=False).update_traces(hovertemplate=None).update_layout(
font={"color": "rgb(64,64,64)",
      "family": "Times New Roman",
      "size": 20},
titlefont={"color": "rgb(64,64,64)",
           "family": "Times New Roman",
           "size": 40}
)

HTML(fig.to_html())
######################

##### Add South Sudan

In [ ]:
# Make South Sudan Points Nuetral
for i in range(0,len(df)):
    if dfss[i:i+1]['ID'][i].startswith('SS') is True:
        dfss["النقاط"][i:i+1] = (df["النقاط"].min() + df["النقاط"].max())/2

In [ ]:
HTML(fig.add_choropleth(
    autocolorscale=False,
    z=dfss["النقاط"],
    locations=dfss["الولاية"],
    locationmode='geojson-id',
    geojson=sudan,
    showscale=False,
    marker=dict(opacity=0.5),
    hoverinfo='location'
).update_traces(name='').to_html())

In [ ]:
# fig.write_html("drive/My Drive/SD_SSD_States_Animation.html")

# --------------------------------------------------------------------------------------------------